In [ ]:
import sys
import os

sys.path.append('scripts')

import re
import json
import argparse
import itertools
from pathlib import Path
import collections
from concurrent.futures import ProcessPoolExecutor


import statistics
from tabulate import tabulate


import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
from matplotlib.patches import PathPatch
from matplotlib.path import get_path_collection_extents
import seaborn as sns
import pandas as pd
import polars as pl

from rich import print, pretty
from typing import  Iterable
import pretty_errors
from catppuccin import PALETTE

print(os.getcwd())

# import .scripts.ldj
from ldj import ldj
from utils import *

pretty.install()

RESULTS_DIRS = [
    Path('./experiments/solo-gp'),
]

for RESULTS_DIR in RESULTS_DIRS:
    assert RESULTS_DIR.is_dir() and RESULTS_DIR.exists()

flavor = PALETTE.latte.colors
# num-robots-10-seed-0.json
RE = re.compile(r"tracking-(.+?)-seed-(\d+).json")


In [ ]:
# use LaTeX for text with matplotlib
sns.set_style("darkgrid")
# set background color of sns darkgrid to flavor.base.hex
plt.rcParams['axes.facecolor'] = flavor.base.hex
# set font color to flavor.text.hex
plt.rcParams['text.color'] = flavor.text.hex

font_dirs = ["./scripts/fonts/"]
# go through all fonts in the font directory and add them
for font_dir in font_dirs:
    for font in os.listdir(font_dir):
        fm.fontManager.addfont(f"{font_dir}/{font}")

prop_jbm = fm.FontProperties(fname='./scripts/fonts/JetBrainsMonoNerdFontMono-Regular.ttf')
prop = fm.FontProperties(fname='./scripts/fonts/STIXTwoText-VariableFont_wght.ttf')

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": prop.get_name(),
    # "font.family": "stix",
    # "font.sans-serif": prop.get_name(),
    "mathtext.fontset": "stix",
    # "text.latex.preamble": r"\usepackage{fontenc}\usepackage{fontspec}\setmainfont{JetBrainsMonoNerdFontMono-Regular}",
})

print(prop.get_name())

colors = [
    (flavor.lavender.hex, 1.0, (1, 0), 'o'),
    (flavor.green.hex, 1.0, (2, 2), 'X'),
]

In [ ]:

# empty polars dataframe with header ["robot_id", "tracking", "seed", "rmse"]
df = pl.DataFrame(schema={
    "robot_id": pl.String,
    "tracking": pl.Boolean,
    "seed": pl.Int64,
    "rmse": pl.Float64
})

for directory in RESULTS_DIRS:

    # rmses: list[float] = []
    rmse_true: dict[str, float] = {}
    rmse_false: dict[str, float] = {}
    # rmse_of_each_robot: dict[str, float] = {}
    for file in RESULTS_DIR.glob('*.json'):
        match = RE.match(file.name)
        assert match is not None
        tracking = str(match.group(1))
        seed = int(match.group(2))
    
        data = json.loads(file.read_text())

        for robot_id, robot_data in data['robots'].items():
            color: str = robot_data['color']
            positions = np.array([p for p in robot_data['positions']])
            mission = robot_data['mission']
            waypoints = []
            for route in mission['routes']:
                waypoints.append(route['waypoints'][0])
                for wp in route['waypoints'][1:]:
                    waypoints.append(wp)

            waypoints = np.array(waypoints)
            waypoints = np.squeeze(waypoints)

            x_coords = []
            y_coords = []
            projection_x_coords = []
            projection_y_coords = []

            lines: list[LinePoints] = [LinePoints(start=start, end=end) for start, end in sliding_window(waypoints, 2)]
            closest_projections = [closest_projection_onto_line_segments(p, lines) for p in positions]

            # plot a solid lines of all waypoints
            plt.plot(waypoints[:, 0], waypoints[:, 1], linestyle='solid', color=color, label=f'Robot {robot_id}')

            # plot a dashed line from each point to closest projection
            for p, cp in zip(positions, closest_projections):
                x_coords.append(p[0])
                y_coords.append(p[1])
                projection_x_coords.append(cp[0])
                projection_y_coords.append(cp[1])

                #plot line to closest projection
                plt.plot([p[0], cp[0]], [p[1], cp[1]], color=color)

            error: float = np.sum(np.linalg.norm(positions - closest_projections, axis=1))
            rmse: float = np.sqrt(error / len(positions))

            # if tracking == 'true':
            #     rmse_true[f"{robot_id}-{tracking}-{seed}"] = rmse
            # else:
            #     rmse_false[f"{robot_id}-{tracking}-{seed}"] = rmse

            df2 = pl.DataFrame({
                "robot_id": [robot_id],
                "tracking": [tracking == "true"],
                "seed": [seed],
                "rmse": [rmse]
            })

            # print(df2)

            # put into dataframe
            df.extend(df2)

df_tracking = df.filter(pl.col("tracking") == True)
df_no_tracking = df.filter(pl.col("tracking") == False)
print(df_tracking)
print(df_tracking['rmse'].describe())
print(df_no_tracking)
print(df_no_tracking['rmse'].describe())
    


In [ ]:

# make box plots

fig, ax = plt.subplots(figsize=(4,2))

offset = 0.2

def set_box_plot_color(bp, color, alpha):
    # style the outliers
    # plt.setp(bp['fliers'], color=color, markersize=3, marker='.')
    # above color is not working, they are all black do this instead:
    for flier in bp['fliers']:
        flier.set(marker='d', markerfacecolor=color, markeredgecolor=None, alpha=0.5 * alpha)

    for element in ['boxes', 'whiskers', 'fliers', 'medians', 'caps']:
        base_alpha = 1.0
        if element == 'boxes':
            base_alpha = 0.75
        elif element == 'fliers':
            base_alpha = 0.25
        plt.setp(bp[element], color=color, alpha=base_alpha * alpha)

for i, (d, (c, alpha, _, _)) in enumerate(zip([df_no_tracking, df_tracking], colors)):
    rmse = d['rmse'].to_numpy()

    bp = ax.boxplot([rmse], positions=[i], widths=0.5, patch_artist=True)
    set_box_plot_color(bp, c, alpha)

ax.set_ylim(-0.2, 1.7)

ax.set_ylabel('RMSE', fontdict={'color': flavor.text.hex})
ax.set_xticks([0, 1])
ax.set_xticklabels(['No Tracking', 'Tracking'], fontdict={'color': flavor.text.hex})

# y ticks
# plt.yticks(color=flavor.text.hex)
ax.set_yticklabels(ax.get_yticks(), fontdict={'color': flavor.text.hex}, fontproperties=prop_jbm)

fig.tight_layout()
fig.savefig('./scripts/out/solo-gp.svg')

fig.show()

In [ ]:
df = pl.DataFrame(schema={
    "robot_id": pl.String,
    "tracking": pl.Boolean,
    "seed": pl.Int64,
    "rmse": pl.Float64,
    "waypoints_key": pl.String,
    "positions_key": pl.String,
})

waypoints_dict = dict[str, np.ndarray]()
positions_dict = dict[str, np.ndarray]()

for directory in RESULTS_DIRS:
    for file in RESULTS_DIR.glob('*.json'):
        # fig, ax = plt.subplots(figsize=(5, 4))
        # # equal axis
        # ax.axis('equal')

        match = RE.match(file.name)
        assert match is not None
        tracking = str(match.group(1))
        seed = int(match.group(2))
    
        data = json.loads(file.read_text())

        for robot_id, robot_data in data['robots'].items():
            color: str = robot_data['color']
            positions = np.array([p for p in robot_data['positions']])
            mission = robot_data['mission']
            waypoints = []
            for route in mission['routes']:
                waypoints.append(route['waypoints'][0])
                for wp in route['waypoints'][1:]:
                    waypoints.append(wp)

            waypoints = np.array(waypoints)
            waypoints = np.squeeze(waypoints)

            waypoints_dict[f"{tracking}-{seed}"] = waypoints
            positions_dict[f"{tracking}-{seed}"] = positions

        #     # plot a solid lines of all waypoints
        #     ax.plot(waypoints[:, 0], waypoints[:, 1], linestyle='dotted', color=color, label=f'Robot {robot_id}')
        #     #plot the robot positions
        #     ax.plot(positions[:, 0], positions[:, 1], color=color)
        
        # ax.set_title(f"Tracking: {tracking}, Seed: {seed}")

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3))
ax.axis('equal')

styles = [
    (flavor.lavender.hex, 1.0, (1, 0), 'o'),
    (flavor.green.hex, 1.0, (2, 2), 'X'),
]

seed = 227
waypoints = waypoints_dict[f"true-{seed}"]
# plot waypoint path as a solid line
ax.plot(waypoints[:, 0], waypoints[:, 1], linestyle='solid', color=flavor.text.hex, label=f"Waypoints {tracking}")

for tracking, (c, a, _, _) in zip(['false', 'true'], styles):
    positions = positions_dict[f"{tracking}-{seed}"]

    # plot
    ax.plot(positions[:, 0], positions[:, 1], color=c, label=f"Robot {tracking}")

# x ticks
ax.set_xticklabels(ax.get_xticks(), fontdict={'color': flavor.text.hex}, fontproperties=prop_jbm)
# x tick formatter to round to zero decimal places
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.0f}"))

# y ticks
ax.set_yticklabels(ax.get_yticks(), fontdict={'color': flavor.text.hex}, fontproperties=prop_jbm)
# y tick formatter to round to zero decimal places
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y:.0f}"))

ax.set_xlim(40, 75)
ax.set_ylim(-30, 30)

fig.tight_layout()
fig.savefig('./scripts/out/solo-gp-deviation.svg')

In [ ]:
p1 = [1.0, 0.0]
p2 = [-1.0, 1.0]
abs( np.arctan2(p1[0], p1[1]) - np.arctan2(p2[0], p2[1]) ) >= np.pi / 2

In [ ]:
lines: List[LinePoints] = [
    LinePoints(start=[0, 0], end=[0, 1]),
    LinePoints(start=[0, 1], end=[0.2, 1.2]),
    LinePoints(start=[0.2, 1.2], end=[1.2, 1.2]),
]

point = np.array([.3, 0.9])

fig, ax = plt.subplots()

# plot point
ax.plot(point[0], point[1], 'o', color='blue')

for line in lines:
    # plot line
    start = line.start
    end = line.end
    ax.plot([start[0], end[0]], [start[1], end[1]], color='black')
    # ax.plot([line.start[0], line.end[0]], [line.start[1], line.end[1]], color='black')

    v1 = point - line.start
    v2 = point - line.end

    # projection onto line
    # line_segment = line_from_line_segment(line.start[0], line.start[1], line.end[0], line.end[1])
    projection = projection_onto_line(point, line)

    # plot projection
    ax.plot(projection[0], projection[1], 'o', color='red')

    # plot vectors
    ax.quiver(*start, *v1, angles='xy', scale_units='xy', scale=1, color='blue')
    ax.quiver(*end, *v2, angles='xy', scale_units='xy', scale=1, color='blue')

# equal axis
ax.axis('equal')

fig.show()


In [ ]:

    
data = json.loads(RESULTS_DIRS[0].read_text())

rmses: list[float] = []
rmse_true: dict[str, float] = {}

for robot_id, robot_data in data['robots'].items():
    color: str = robot_data['color']
    positions = np.array([p for p in robot_data['positions']])
    mission = robot_data['mission']
    waypoints = []
    for route in mission['routes']:
        waypoints.append(route['waypoints'][0])
        for wp in route['waypoints'][1:]:
            waypoints.append(wp)

    # print(waypoints)

    # waypoints = np.array([wp for wp in (route['waypoints'] for route in mission['routes'])])
    waypoints = np.array(waypoints)

    waypoints = np.squeeze(waypoints)

    x_coords = []
    y_coords = []
    projection_x_coords = []
    projection_y_coords = []

    lines: list[Line] = [line_from_line_segment(*start, *end) for start, end in sliding_window(waypoints, 2)]
    closest_projections = [closest_projection_onto_line_segments(p, lines) for p in positions]

    # plot a solid lines of all waypoints
    # plt.plot(waypoints[:, 0], waypoints[:, 1], linestyle='solid', color=color)
    plt.plot(waypoints[:, 0], waypoints[:, 1], linestyle='solid', color=color, label=f'Robot {robot_id}')

    # plot a dashed line from each point to closest projection
    for p, cp in zip(positions, closest_projections):
        # plt.plot([p[0], cp[0]], [p[1], cp[1]], linestyle='dashed', color='red')
        x_coords.append(p[0])
        y_coords.append(p[1])
        projection_x_coords.append(cp[0])
        projection_y_coords.append(cp[1])

    error: float = np.sum(np.linalg.norm(positions - closest_projections, axis=1))
    rmse: float = np.sqrt(error / len(positions))

    for ((p1, cp1), (p2, cp2)) in sliding_window(zip(positions, closest_projections), 2):
        # plt.fill([p1[0], cp1[0], cp2[0], p2[0]], [p1[1], cp1[1],  cp2[1], p2[1]], color=color, alpha=0.3)
        # plt.fill([p])
        xs = [p1[0], p2[0], cp1[0], cp2[0]]
        ys = [p1[1], p2[1], cp1[1], cp2[1]]
        # xs = [p2[0], p1[0], cp2[0], cp1[0]]
        # ys = [p2[1], p1[1], cp2[1], cp1[1]]
        # xs = [cp1[0], cp2[0], p2[0], p1[0]]
        # ys = [cp1[1], cp2[1], p2[1], p1[1]]
        # xs = [cp2[0], cp1[0], p2[0], p1[0]]
        # ys = [cp2[1], cp1[1], p2[1], p1[1]]
        # xs = [cp2[0], cp1[0], p2[0], p1[0]]
        # ys = [cp2[1], cp1[1], p2[1], p1[1]]
        plt.fill(xs, ys, color=color, alpha=0.3)



    # print(f"{robot_id} RMSE: {rmse:.3f}")
    rmse_true[robot_id] = rmse

mean: float = statistics.mean(rmse_true.values())
median: float = statistics.median(rmse_true.values())
largest: float = max(rmse_true.values())
smallest: float = min(rmse_true.values())
variance: float = statistics.variance(rmse_true.values())
stdev: float = statistics.stdev(rmse_true.values())
N: int = len(rmse_true)

stats = dict(
    robots=N,
    mean=mean,
    median=median,
    largest=largest,
    smallest=smallest,
    variance=variance,
    stdev=stdev
)

headers = ['Robot ID', 'RMSE']
table = [[robot_id, f"{rmse:.3f}"] for robot_id, rmse in rmse_true.items()]
tabulate_opts = dict(
    tablefmt="mixed_outline",
    showindex="always"
)
# print(tabulate(table, headers, tablefmt="mixed_outline"))
print(tabulate(table, headers, **tabulate_opts))

print(tabulate([stats], headers="keys", **tabulate_opts))



plt.grid(True)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()